In [1]:
from gpiozero import RotaryEncoder, Button
from time import sleep
import threading
from TGS2620Sensor import TGS2620Sensor
from MQ135Sensor import MQ135Sensor
from MAX30102Sensor import MAX30102Sensor
from MPU6050Sensor import MPU6050Sensor
from CameraSensor import CameraSensor

In [2]:
encoder = RotaryEncoder(a=6, b=5)  # CLK连接GPIO6，DT连接GPIO5
button = Button(12)                  # SW连接 GPIO12
globalCounter = 0          # 计数器值
tmp = 0	                # 当前状态判断

# 模式列表
TGS2620 = TGS2620Sensor()
MQ135 = MQ135Sensor()
MAX30102 = MAX30102Sensor()
MPU6050 = MPU6050Sensor()
CAMERA = CameraSensor()

def do_nothing():
    pass

modes = ["CAMERA","MPU6050", "MAX30102", "MQ135", "TGS2620", "Start All", "Shut Down All"]
start_modes = ["Start CAMERA","Start MPU6050", "Start MAX30102", "Start MQ135", "Start TGS2620", "Start All", "Shut Down All"]
run_modes = [CAMERA.run, MPU6050.run, MAX30102.run, MQ135.run, TGS2620.run, do_nothing, None]
stop_modes = [CAMERA.stop, MPU6050.stop, MAX30102.stop, MQ135.stop, TGS2620.stop, None, do_nothing]
threads = {}

# 旋转编码方向位判断函数
def rotaryDeal():
    global globalCounter
    global tmp
    globalCounter += encoder.steps  # 根据编码器步骤调整计数器
    encoder.steps = 0 # 更新计数器后重置编码器步骤
    if tmp != globalCounter: # 判断状态值发生改变
        mode_index = globalCounter % len(modes)
        print (f"Mode: {modes[mode_index]}") # 打印出状态信息
        tmp = globalCounter    #  把当前状态赋值到下一个状态，避免重复打印

# 中间按键按下响应程序
def btnISR():
    mode_index = globalCounter % len(start_modes)
    # print(f"Confirm Mode: {start_modes[mode_index]}")
    sleep(0.5)

    if stop_modes[mode_index] == None:
        for i in range(0, len(modes)-2):
            if i not in threads or not threads[i].is_alive():
                thread = threading.Thread(target=run_modes[i], daemon=True)
                threads[i] = thread
                thread.start()
                print(start_modes[i])
                sleep(0.5)            
    elif run_modes[mode_index] == None:
        for i in range(0, len(modes)-2):
            if i in threads and threads[i].is_alive():
                print(f"{modes[i]} is already running")
                stop_modes[i]()
                threads[i].join()
                print(f"{modes[i]} killed")
                sleep(0.5) 
    else:
        if mode_index not in threads or not threads[mode_index].is_alive():
            thread = threading.Thread(target=run_modes[mode_index], daemon=True)
            threads[mode_index] = thread
            thread.start()
            print(start_modes[mode_index])
            sleep(0.5)
        else:
            print(f"{modes[mode_index]} is already running")
            stop_modes[mode_index]()
            threads[mode_index].join()
            print(f"{modes[mode_index]} killed")
            sleep(0.5)

# 按键按下中断响应程序
button.when_pressed = btnISR

Image(value=b'', format='jpeg', height='480', width='640')

[0:10:05.505027540] [3919]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:10:05.514140013] [3959]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[0:10:05.524189025] [3959]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/imx708@1a to CFE device /dev/media0 and ISP device /dev/media2 using PiSP variant BCM2712_C0
[0:10:05.526917040] [3919]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format Y16 
[0:10:05.526949471] [3919]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format RGB6
[0:10:05.526954212] [3919]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format BGR6
[0:10:05.526959991] [3919]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format PC1M
[0:10:05.527636771] [3919]  INFO Camera camera.cpp:1183 configuring streams: (0) 640x480-XRGB8888 (1) 1536x864-GRBG16_PISP_COMP1
[0:10:05.527746935] [3959]  INFO RPI pisp.cpp:1396 Sensor: /base/axi/pcie@120000

In [3]:
if __name__ == '__main__':
    try:
        while True:
            rotaryDeal()  # 更新编码器值
            sleep(0.01)             # 短延迟，降低CPU负载
    except KeyboardInterrupt:
        pass

Mode: MPU6050
Mode: MAX30102
Mode: MQ2
Mode: TGS2620
Mode: Start All
Mode: Shut Down All
Mode: CAMERA
Start CAMERA


Exception in thread Thread-11 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/makerobo_code/sensor_code/control/CameraSensor.py", line 209, in run
    self.update_servo(x, y, w, h)
  File "/home/pi/makerobo_code/sensor_code/control/CameraSensor.py", line 103, in update_servo
    self.kit.servo[10].angle = 220 - self.pan
    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/adafruit_motor/servo.py", line 137, in angle
    raise ValueError("Angle out of range")
ValueError: Angle out of range


Blendshapes data saved to data/Camera.csv
